In [ ]:
# default_exp judge

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export
import heapq
import numpy as np

from typing import Any, List

from nltk.lm import MLE
from nltk.util import ngrams
from nltk.lm.preprocessing import flatten
from nltk.lm.preprocessing import pad_both_ends

from forma.utils import PatternGenerator

# Judge

> Assesses a tabular dataset for format errors.

In [ ]:
# export
class FormatJudge:
    """Detects format errors on a tabular data set."""
    
    def __init__(self, generator: PatternGenerator, n: int = 3, dim: int = 1):
        self.generator = generator
        self.n = n
        self.dim = dim
        
    def __call__(self, o: Any) -> list:
        return self.judge(o)
    
    def fit(self, values: List[Any]):
        patterns = [self.generator(v) for v in values]
        padded_patterns = [pad_both_ends(p, n=self.n) for p in patterns]
        ngrams_ = [ngrams(pp, n=self.n) for pp in padded_patterns]
        
        self.vocab = list(flatten(pad_both_ends(p, n=self.n) for p in patterns))
        self.model = MLE(self.n)
        self.model.fit(ngrams_, self.vocab)
        
    def judge(self, o: Any) -> list:
        scores = []
        p = self.generator(o)
        p = list(pad_both_ends(p, n=self.n))
        for i,v in enumerate(p):
            if i < self.n-1:
                continue
            letters = []
            for j in range(i - (self.n-1), i):
                letters.append(p[j])
            scores.append(self.model.score(v, letters))
        return heapq.nsmallest(self.dim, scores)